<a href="https://colab.research.google.com/github/aubreymoore/crb-damage-detector-colab/blob/main/detect_and_annotate_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# detect_and_annotate.ipynb

NOTE: The following documentation is already slightly out of date.
Please visit https://github.com/aubreymoore/crb-damage-detector-colab before running this notebook for the first time.

This Colab Jupyter notebook runs a custom YOLOv8 object detector which scans images to find three object classes: live coconut palms, dead coconut palms and v-shaped cuts symptomatic of damage caused by coconut rhinoceros beetle, *Oryctes rhinoceros*.

IMPORTANT: Shortly after the MAIN PROGRAM section begins executing, a BROWSE button will appear below the active cell to allow you to upload single file of input data from your loacal machine to Colab.

**Note that Colab will just sit there and not do anything until you have entered a path to a test file of URLs or a ZIP file of images on your local machine.** [Click here to scroll down to the "Browser" button.](#scrollTo=5zSjfTXvIv2q&line=1&uniqifier=1)

You may choose between 2 options:
* A TEXT file (\*.txt) containing URLs for images to be scanned. One URL per line. (This is the most efficient option.)
* A ZIP file (\*.zip) containing images to be scanned.


Test data are available in a companion GitHub repository at https://github.com/aubreymoore/crb-damage-detector-colab. To use the test data, download it to your local computer it as a [ZIP file](https://github.com/aubreymoore/crb-damage-detector-colab/archive/refs/heads/main.zip) and unzip it. If you have **git** installed, you can clone the repo as an alternative. The TEXT file or ZIP file to be uploaded to Colab will be found in the repository's **data** folder.

To scan images, select **Runtime | Run all** on the main menu.
Results will be in a temporary OUTPUT folder which you can access using the **File browser** in the left Colab panel.

When image scanning is complete, the OUTPUT folder will be compressed into a single ZIP file and automatically downloaded to your computer.

### TODO

- [ finished 2024-10-19] Reduce size of images in the companion GH repo to max dimension of 960px
- [ ] Copy current trained model to companion GH repo
- [ ] Copy this Jupyter notebook to companion GH repo
- [ ] Add confidence values to bounding box labels.
- [ ] Add database to OUTPUT folder
- [ ] Extract GPS coordinates from image files
- [ ] Figure out how to use URLs to access images stored on OneDrive (Sharepoint)

# Load Python packages which are not preinstalled by Colab

In [ ]:
%pip install ultralytics -q
%pip install supervision -q
%pip install imutils -q
%pip install icecream -q
%pip install ipython-autotime -q

# Import modules

In [ ]:
import cv2
import supervision as sv
from ultralytics import YOLO
import imutils
import glob
import os
import shutil
from skimage import io
from icecream import ic
from google.colab import files
# ultralytics.checks()

# Load cell timer

In [ ]:
%load_ext autotime

# Define functions

In [ ]:
from urllib.request import urlretrieve

url = 'https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/images/IMG_0532.JPG?raw=true'
filename = 'IMG_0532.JPG'
urlretrieve(url, filename)

In [ ]:
def get_gps_from_exif(image_path):
  """
  Gets timestamp and GPS coordinates from an image.

  Args:
    image_path:

  Returns:
    timestamp, latitude, longitude
  """
  with open(image_path, 'rb') as src:
    img = Image(src)
    if img.has_exif:
      try:
        timestamp = img.datetime_original
        print(img.gps_latitude)
        dms = img.gps_latitude
        latitude = dms[0] + dms[1]/60 + dms[2]/3600
        if img.gps_latitude_ref == 'S':
          latitude = -latitude
        dms = img.gps_longitude
        longitude  = dms[0] + dms[1]/60 + dms[2]/3600
        if img.gps_longitude_ref == 'W':
          longitude = -longitude
        return {"timestamp": timestamp, "latitude": latitude, "longitude": longitude}
      except Exception as e:
        print(e)
        return {"timestamp": None, "latitude": None,"longitude": None}
    else:
      print ('The Image has no EXIF')
      return {"timestamp": None, "latitude": None,"longitude": None}

get_gps_from_exif('IMG_0532.JPG')

In [ ]:
def upload_model_weights():
  '''
  Upload model weights from GitHub repo to **weights.pt** only if this file does not already exist.
  '''
  !wget -nc https://github.com/aubreymoore/code-for-CRB-damage-ai/raw/refs/heads/main/models/3class/train5/weights/best.pt -O weights.pt

# upload_model_weights()

In [ ]:
def load_model_weights():
  model = YOLO('weights.pt')

In [ ]:
def create_input_folder():
  if not os.path.exists('INPUT'):
    os.makedirs('INPUT')

# create_input_folder()

In [ ]:
def create_output_folder():
  if not os.path.exists('OUTPUT'):
    os.makedirs('OUTPUT')

# create_output_folder()

In [ ]:
def run_garbage_disposal():
  '''
  Delete any data files left over from the last run.
  '''
  shutil.rmtree('INPUT', ignore_errors=True)
  shutil.rmtree('OUTPUT', ignore_errors=True)
  shutil.rmtree('sample_data', ignore_errors=True)

  try:
    os.remove('weights.pt')
  except OSError:
    pass

# run_garbage_disposal()

In [ ]:
def extract_JPG_files(zip_file_path, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Loop through each file in the zip archive
        for file_name in zip_ref.namelist():
            if file_name.endswith('.JPG'):  # Check for .jpg extension
                print(f'Extracting {file_name}...')
                zip_ref.extract(file_name, output_dir)  # Extract the file

# Usage
# extract_dll_files('path/to/your/archive.zip', 'path/to/extract/directory')


In [ ]:
import zipfile
import io

def extract_zip_to_memory(zip_content):
    """
    Extracts files from a ZIP archive stored in memory.

    :param zip_content: Bytes of the ZIP file.
    :return: A dictionary of filename and file-like objects.
    """
    extracted_files = {}
    with zipfile.ZipFile(io.BytesIO(zip_content)) as z:
        for file_info in z.infolist():
            with z.open(file_info) as file:
                extracted_files[file_info.filename] = file.read()  # Read file content
    return extracted_files

# Usage example
# Assuming 'zip_data' contains the bytes of your ZIP file
# zip_data = ... (load your ZIP data here)
# files = extract_zip_to_memory(zip_data)
# for name, content in files.items():
#     print(f"Extracted {name} with size {len(content)} bytes")


In [2]:
def upload_and_unpack_zip_or_txt():
  '''
  Upload images in a ZIP (*.zip) or list of URLs (*.txt)


  '''
  input_mode = None
  urls = None
  image_file_dir = None

  # Upload images in a ZIP (*.zip) or list of URLs (*.txt)

  uploaded = files.upload(target_dir='INPUT')
  filename = list(uploaded.keys())[0]

  if filename.endswith('.txt'):
    input_mode = 'text'
    with open(filename, 'r') as f:
      urls = f.read().splitlines()

      for url in urls:

        # Extract filename from URL
        filename = url.split('/')[-1]
        pos = filename.find('?')
        if pos >= 0:
          filename = filename[:pos]

        urlretrieve(url, f'INPUT/filename')

  elif filename.endswith('.zip'):
    input_mode = 'zip'
    !unzip -q $filename -d INPUT
    image_file_dir = f'INPUT/{filename}'.replace('.zip', '')
    ic(image_file_dir)

  else:
    raise ValueError('INPUT file must be *.txt or *.zip.')
  return input_mode, urls, image_file_dir

# input_mode, urls, image_file_dir = upload_and_unpack_zip_or_txt()
# ic(input_mode)
# ic(urls)
# ic(image_file_dir)

In [ ]:
def get_input_file_list():
  return glob.glob(f'INPUT/**/*', recursive=True)

# get_input_file_list()

In [ ]:
def detect_objects(image, model, box_annotator, label_annotator, csv_sink):
  '''
  detect objects in an image
  returns detections and an annotated image
  '''
  results = model(image)[0]
  detections = sv.Detections.from_ultralytics(results)
  # ic(detections)
  annotated_image = box_annotator.annotate(image, detections=detections)
  labels = [f"{model.model.names[class_id]} {confidence:.2f}" for class_id, confidence in zip(detections.class_id, detections.confidence)]
  annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)
  return detections, annotated_image

# csv_sink = sv.CSVSink('detections.csv')
# csv_sink.open()

# upload_model_weights()
# model = YOLO('weights.pt')
# box_annotator = sv.BoxAnnotator()
# label_annotator = sv.LabelAnnotator()

# url = 'https://github.com/aubreymoore/crb-damage-detector-colab/blob/main/data/Vanuatu_July_2022_Sulav/resized-images/IMG_0532.JPG?raw=true'
# image = imutils.url_to_image(url)
# detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink)
# ic(detections)
# sv.plot_image(annotated_image)

# custom_data = {'url': url}
# csv_sink.append(detections, custom_data)

# csv_sink.close()

# MAIN PROGRAM

In [ ]:
# Clear data files from previous run
run_garbage_disposal()

create_input_folder()
create_output_folder()

# Upload images or list of URLs
input_mode, urls, image_file_dir = upload_and_unpack_zip_or_txt()

# Upload weights from trained model and load them
upload_model_weights()
model = YOLO('weights.pt')

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()
csv_sink = sv.CSVSink('OUTPUT/detections.csv')
csv_sink.open()

# Scan images
if input_mode == 'text':
  for url in urls:
    try:
      # image = imutils.url_to_image(url)





      image = cv2.imread('image.jpg')
      detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink)
      csv_sink.append(
          detections,
          custom_data={'image_h': image.shape[0], 'image_w': image.shape[1], 'source': url}
      )

      # Extract filename from URL
      filename = url.split('/')[-1]
      pos = filename.find('?')
      if pos >= 0:
        filename = filename[:pos]

      output_path = f'OUTPUT/{filename}'.replace('.', '_annotated.')
      ic(output_path)
      os.makedirs(os.path.dirname(output_path), exist_ok = True)
      cv2.imwrite(output_path, annotated_image)
    except:
      print(f'Error processing {url}')
    continue

if input_mode == 'zip':
  input_file_list = get_input_file_list()
  ic(input_file_list)
  for image_path in input_file_list:
    ic(image_path)
    try:
      image = cv2.imread(image_path)
      detections, annotated_image = detect_objects(image, model, box_annotator, label_annotator, csv_sink)
      csv_sink.append(
          detections,
          custom_data={'image_h': image.shape[0], 'image_w': image.shape[1], 'source': image_path}
      )

      filename = os.path.basename(image_path)
      output_path = f'OUTPUT/{filename}'.replace('.', '_annotated.')
      os.makedirs(os.path.dirname(output_path), exist_ok = True)
      result = cv2.imwrite(output_path, annotated_image)
    except:
      print(f'Error processing {image_path}')
    continue

csv_sink.close()

## Please click on the Browse buttom when it appears above this cell.

### Download OUTPUT folder as a ZIP file

In [ ]:
!zip -r OUTPUT.zip OUTPUT

In [ ]:
from google.colab import files
files.download("OUTPUT.zip")

# FINISHED
If everything worked as intended, you should find a file named **OUTPUT.zip** in your Downloads folder. Unzip this file to see results.

In [ ]:

print('FINISHED')